In [ ]:
import torch
from transformers import pipeline
from transformers import BertTokenizer, AutoModel
import pandas as pd
import random

In [ ]:
# The pipeline('fill-mask') automatically loads a pretrained masked-language model.
# BERT predicts the missing [MASK] token
# result contains a list of top token predictions with probabilities.

unmasker = pipeline('fill-mask', model='bert-base-uncased')

test = "Every morning last summer in Greece, " + \
       "I visited the [MASK] where I would swim, " + \
       "play in the sand, and sunbathe."

result = unmasker(test)
result

## Sentiment Data

In [ ]:
url = 'https://www.dropbox.com/scl/fi/i2esmtinb4qor0mzokybp/fed_sentiment_training.csv?rlkey=v9u7afunmy8w0v0lwizba5g25&dl=1'
df = pd.read_csv(url, sep='\t')

In [ ]:
df.head(10)

In [ ]:
df.loc[0, "text"]

### Tokenizing Text

In [ ]:
# Loads the same tokenizer that BERT was trained with.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer # [PAD] is used to pad the text to the max length 512

In [ ]:
# The tokenizer’s vocabulary contains about 30,000 subword tokens.
# Subword tokens look like un, ##happy, etc.
# Each has a unique integer ID that BERT uses internally.

vocab = tokenizer.get_vocab()
print(f"Total number of tokens in vocabulary: {len(vocab)} \n---------")
for _ in range(10): # Repeats the process 10 times but I am not going to use the loop variable
    word, idx = random.choice(list(vocab.items()))
    print(word, idx)

In [ ]:
# takes one text string (the first row) and converts it into tensors suitable for BERT.
encoded_input1 = tokenizer(df.loc[0, "text"],
                           max_length=100,
                           padding="max_length",
                           return_tensors='pt')

In [ ]:
encoded_input1

In [ ]:
# Each number corresponds to a token ID in BERT’s 30k-word vocabulary.
# tensor of shape [1, 100]
encoded_input1.input_ids

In [ ]:
# This tells BERT which tokens are real and which are padding.
# 1 → real token (to be attended to)
# 0 → padding token (ignore it)

encoded_input1.attention_mask

In [ ]:
# 1D tensor of shape [100]
temp_tokens = encoded_input1["input_ids"][0]  # ← Add [0] here to get first sequence
print("Tokens IDs:")
print(temp_tokens)

In [ ]:
# translates those integer IDs back into their string tokens — i.e., the subwords that BERT uses internally.
print("\n------------------------------------------\n")
print("Tokens:")
print(tokenizer.convert_ids_to_tokens(temp_tokens))

### Obtaining Embeddings

In [ ]:
# Loads the pretrained BERT model weights.
# Each input token will be represented by a 768-dimensional embedding at each layer.
model = AutoModel.from_pretrained("bert-base-uncased",
                                  output_hidden_states=True, # keeps internal layer outputs.
                                  output_attentions=True, # keeps attention matrices
                                  attn_implementation="eager"
                                  )

print(model.config)

In [ ]:
# Step 1: Get BERT output without computing gradients (inference mode)
# Disables gradient tracking (saves memory & compute). Normally, PyTorch builds
# a computation graph during forward passes so it can later compute gradients (for training).
with torch.no_grad():
    result1 = model(**encoded_input1)
    # The ** operator unpacks the dictionary into keyword arguments
    # ** (double star) → unpacks a dictionary into keyword arguments

In [ ]:
def greet(name, age):
    print(f"Hello {name}, you are {age} years old.")
person = {'name': 'Alice', 'age': 25}
greet(**person)

In [ ]:
# Step 2: Extract token-level embeddings from BERT's last layer
last_hidden_state = result1.last_hidden_state
print(f"Token embeddings shape: {last_hidden_state.shape}")

In [ ]:
# Step 3: Get attention mask (1 = real token, 0 = padding)
attention_mask = encoded_input1["attention_mask"]  # [1, 30]
print(f"Attention mask shape: {attention_mask.shape}")
print("\n------------------------------------------\n")
print("attention mask:")
print(attention_mask)
print("\n------------------------------------------\n")

In [ ]:
# Step 4: Zero out padding token embeddings
# unsqueeze(-1) adds dimension: [1, 100] → [1, 100, 1]
# This allows broadcasting when multiplying with embeddings [1, 100, 768]
masked_embeddings = last_hidden_state * attention_mask.unsqueeze(-1)

In [ ]:
# Step 5: Compute mean pooling (average of non-padding tokens)
# Numerator: sum all 30 token embeddings
sum_embeddings = masked_embeddings.sum(dim=1)  # [1, 768]

In [ ]:
# Denominator: count how many real tokens
num_real_tokens = attention_mask.sum(dim=1, keepdim=True)  # [1, 1]

In [ ]:
# Final sentence embedding: average of real token embeddings
mean_embedding1 = sum_embeddings / num_real_tokens  # [1, 768]

In [ ]:
print(f"Sentence embedding shape: {mean_embedding1.shape}")
print("\n------------------------------------------\n")
print("First Ten Elements of Embedding:")
print(mean_embedding1[0, :10])

In [ ]:
# %% Now scale to ALL examples in the dataset
import numpy as np

# Move model to GPU
model = model.to('cuda')

batch_size = 32  # Process 32 texts at once
all_embeddings = []

for i in range(0, len(df), batch_size):
    batch_texts = df["text"][i:i+batch_size].tolist()

    # Tokenize the batch
    encoded_input = tokenizer(batch_texts,
                             max_length=30, # for speed
                             padding="max_length",
                             truncation=True,
                             return_tensors='pt').to('cuda')  # Move batch to GPU

    with torch.no_grad():
        result = model(**encoded_input)

    last_hidden_state = result.last_hidden_state
    attention_mask = encoded_input["attention_mask"]
    masked_embeddings = last_hidden_state * attention_mask.unsqueeze(-1)
    mean_embedding = masked_embeddings.sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

    # Move back to CPU and convert to numpy
    batch_embeddings = mean_embedding.cpu().numpy()
    all_embeddings.extend(batch_embeddings)

# stack into matrix
embeddings_matrix = np.vstack(all_embeddings)

print(f"Embeddings matrix shape: {embeddings_matrix.shape}")
print(f"Number of texts: {embeddings_matrix.shape[0]}")
print(f"Embedding dimension: {embeddings_matrix.shape[1]}")

# Store in dataframe
df['embedding'] = all_embeddings

In [ ]:
df.head()